In [33]:
using Surrogates
using Plots
using Statistics
using Random
using DataFrames
using Distances
using LinearAlgebra
default()

In [95]:
# Define the objective waterflow function (8 dimensions)
function f(x)
    r_w = x[1]
    r = x[2]
    T_u = x[3]
    H_u = x[4]
    T_l = x[5]
    H_l = x[6]
    L = x[7]
    K_w = x[8]
    log_val = log(r/r_w)
    return (2*pi*T_u*(H_u - H_l))/ ( log_val*(1 + (2*L*T_u/(log_val*r_w^2*K_w)) + T_u/T_l))
end


n = 250
initial_n = 50
d = 8
lb = [0.05,100,63070,990,63.1,700,1120,9855]
ub = [0.15,50000,115600,1110,116,820,1680,12045]
original_x = sample(initial_n,lb,ub,SobolSample())
original_y = f.(original_x)

50-element Vector{Float64}:
  16.614887299231814
  66.71434150808243
 145.01472282611837
  50.379068168663856
  45.72674669008426
 218.59877093618695
  88.33929805974748
  31.64730062042864
  37.60355111410419
 141.98371486579754
 146.85835662067186
  43.9988868435236
  38.084914505299544
   ⋮
  80.35171582280238
  50.983847087634985
  40.289796197934024
  84.18332252527621
 132.0679010627322
  78.6406514896613
  46.946759319980025
 161.65406427158032
  52.79670900960365
  26.784637013946686
  33.81519848960852
 115.332266280138

In [3]:
function splitdf(df, pct)
           @assert 0 <= pct <= 1
           ids = collect(axes(df, 1))
           shuffle!(ids)
           sel = ids .<= nrow(df) .* pct
           return DataFrame(view(df, sel, :)), DataFrame(view(df, .!sel, :))
       end

splitdf (generic function with 1 method)

In [24]:
# Implementing diversity
function calculate_variance(x, models)
    predictions = []
        for model in models
            prediction = model(x)
            append!(predictions, prediction)
        end
    return var(predictions, corrected=false)
end

function diversity_metric(prev_x, new_x, models, lambda = 0.5, mod)
    min_dist = Inf
    variance = calculate_variance(new_x, models)
    for point in prev_x
        new_dist = euclidean(point, new_x)
        if new_dist < min_dist
            min_dist = new_dist
        end
    end
    return (1 - lambda) * sqrt(variance) + lambda * min_dist
end

diversity_metric (generic function with 2 methods)

In [84]:
function calculate_error(point, models, actual, mode="MSE")
    target = actual(point)
    errors = []
    for model in models
        prediction = model(point)
        append!(errors, abs(target - prediction))
    end
    println(errors)
    if mode == "MSE"
        return mean(errors.^2)
    end
    if mode == "max"
        return maximum(errors)
    end
end

calculate_error (generic function with 2 methods)

In [96]:
total_samples = 10000
prev_points = copy(original_x)
y = copy(original_y)
train = sample(total_samples, lb, ub, SobolSample())
test = sample(200, lb, ub, GoldenSample())
y_true = f.(test)
error = Inf
while error > 2
    kriging_surrogate = Kriging(prev_points, y, lb, ub, p=[1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9])
    my_radial_basis = RadialBasis(prev_points, y, lb, ub)
    x = []
    max_score = 0
    max_index = -1
    for j in 1:length(train)
        point = train[j]
        score = diversity_metric(prev_points, point, [my_radial_basis, kriging_surrogate], 0)
        if score > max_score
            max_score = score
            max_index = j
        end
    end
    append!(x, train[[max_index]])
    deleteat!(train, max_index)
    prev_points = vcat(prev_points, x)
    y = f.(prev_points)
    
    y_rad = my_radial_basis.(test)
    y_krig = kriging_surrogate.(test)
    mse_rad = norm(y_true - y_rad, 2)/length(test)
    mse_krig = norm(y_true - y_krig, 2)/length(test)
    error = (mse_rad + mse_krig) / 2
    println(error)
end

3.430964349159185
3.4172357247317358
3.448074725356175
3.442402333571565
3.4322257352803573
3.430920220617081
3.4254715088837715
3.4222121777743473
3.4283140743535756


LoadError: InterruptException:

In [78]:
prev_points

81-element Vector{Any}:
 (0.0546875, 13354.6875, 100005.15625, 1055.625, 70.5390625, 810.625, 1566.25, 11326.40625)
 (0.10468749999999999, 38304.6875, 73740.15625, 995.625, 96.9890625, 750.625, 1286.25, 10231.40625)
 (0.1296875, 879.6875, 113137.65625, 1085.625, 110.2140625, 720.625, 1426.25, 10778.90625)
 (0.0796875, 25829.6875, 86872.65625, 1025.625, 83.7640625, 780.625, 1146.25, 11873.90625)
 (0.0921875, 7117.1875, 67173.90625, 1040.625, 77.1515625, 795.625, 1496.25, 10505.15625)
 (0.1421875, 32067.1875, 93438.90625, 1100.625, 103.6015625, 735.625, 1216.25, 11600.15625)
 (0.1171875, 19592.1875, 80306.40625, 1010.625, 90.3765625, 705.625, 1636.25, 11052.65625)
 (0.0671875, 44542.1875, 106571.40625, 1070.625, 63.9265625, 765.625, 1356.25, 9957.65625)
 (0.0734375, 3998.4375, 77023.28125, 1108.125, 100.2953125, 788.125, 1531.25, 10642.03125)
 (0.12343749999999999, 28948.4375, 103288.28125, 1048.125, 73.8453125, 728.125, 1251.25, 11737.03125)
 (0.1484375, 16473.4375, 63890.78125, 1078.12

In [57]:
println(size(prev_points))

(51,)


In [93]:
n_test = 200
x_test = sample(n_test,lb,ub,GoldenSample());
y_true = f.(x_test);
my_rad = RadialBasis(prev_points, y, lb, ub)
my_krig = Kriging(prev_points, y, lb, ub, p=[1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9, 1.9])
y_rad = my_rad.(x_test)
y_krig = my_krig.(x_test);

In [94]:
mse_rad = norm(y_true - y_rad,2)/n_test
mse_krig = norm(y_true - y_krig,2)/n_test
print("MSE RadialBasis: $mse_rad    ")
print("MSE Kriging: $mse_krig    ")

MSE RadialBasis: 3.332413447008887    MSE Kriging: 3.257493137405216    